In [1]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import numpy as np
import glob
import keras
from keras.preprocessing.image import smart_resize
from PIL import Image
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D


In [2]:
from google.colab import drive 
drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


In [3]:
folder='/content/drive/MyDrive/CS523_Deep_Learning/Data/train/c'

filelist = glob.glob(folder + '0' + '/*.jpg')
train_images = np.array([np.array(Image.open(fname)) for fname in filelist])
train_labels = np.zeros((train_images.shape[0],), dtype=int)

for i in range(1,10):
    folder_i = folder+str(i)
    filelist = glob.glob(folder_i + '/*.jpg')
    temp=np.array([np.array(Image.open(fname)) for fname in filelist])
    train_images = np.vstack((train_images, temp))
    train_labels = np.hstack((train_labels, np.zeros((temp.shape[0],), dtype=int) + i))


In [10]:
print(train_images.shape)
print(train_labels.shape)
#print(train_images[0])

(20405, 480, 640, 3)
(20405,)


In [5]:
folder='/content/drive/MyDrive/CS523_Deep_Learning/Data/TestData/c'

filelist = glob.glob(folder + '0' + '/*.jpg')
test_images = np.array([np.array(Image.open(fname)) for fname in filelist])
test_labels = np.zeros((test_images.shape[0],), dtype=int)

for i in range(1,10):
    folder_i = folder+str(i)
    filelist = glob.glob(folder_i + '/*.jpg')
    temp=np.array([np.array(Image.open(fname)) for fname in filelist])
    test_images = np.vstack((test_images, temp))
    test_labels = np.hstack((test_labels, np.zeros((temp.shape[0],), dtype=int) + i))

In [6]:
print(test_images.shape)
print(test_labels.shape)

(2019, 480, 640, 3)
(2019,)


In [8]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(480, 640, 3))

base_model.trainable = False
#inputs = base_model.input
inputs = keras.Input(shape=(480, 640, 3))
#inputs = tf.keras.layers.Reshape((60,80,3), input_shape=(480,640,3))(inputs)

scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
inputs = scale_layer(inputs)

#x = base_model.output
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)

predictions = Dense(10, activation='softmax')(x)

model = Model(inputs, outputs=predictions)

#model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

model.summary()

model.fit(train_images, train_labels, epochs=1, validation_data=(test_images, test_labels), batch_size=32)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 480, 640, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, 13, 18, 2048)      21802784  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 1024)              2098176   
                                                                 
 dense_3 (Dense)             (None, 10)                10250     
                                                                 
Total params: 23,911,210
Trainable params: 2,108,426
Non-trainable params: 21,802,784
_______________________________________

InternalError: ignored